# Prediction of Traffic Accident Risk
Team 7: 어서오십쇼HUMAN  
Editor: 
## 2. 모델 학습

In [1]:
# 사전 준비
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

plt.style.use('seaborn')
sns.set(font_scale=2.5)

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sample = pd.read_csv('../input/sample_accident.csv')

print("sample.shape:", sample.shape)

sample.shape: (362297, 10)


In [3]:
sample

,발생일,발생시간,요일,발생지_시군구,사고내용,노면상태,기상상태,도로형태_대분류,가해자차종,가해자연령
0,20100101,00시,금,마포구,경상,젖음/습기,맑음,단일로,승용차,54세
1,20100101,00시,금,동작구,중상,서리/결빙,맑음,단일로,승합차,57세
2,20100101,00시,금,관악구,경상,서리/결빙,흐림,단일로,승용차,51세
3,20100101,00시,금,강서구,경상,건조,맑음,교차로,승용차,56세
4,20100101,00시,금,구로구,경상,건조,맑음,교차로,승용차,33세
...,...,...,...,...,...,...,...,...,...,...
362292,20181231,23시,월,동작구,경상,건조,맑음,단일로,승용차,34세
362293,20181231,23시,월,강서구,부상신고,건조,맑음,교차로,승용차,56세
362294,20181231,23시,월,강동구,경상,건조,맑음,교차로,승용차,57세
362295,20181231,23시,월,강동구,경상,건조,맑음,단일로,승용차,59세


# Prepare Model training and analysis

> editor: seongminyoo   
> Date of modification: 2020-05-23 09:41pm

`solving problem`
+ predict probability

`using model`
+ LinearRegression
+ LogisticRegression
+ Neural Network Regression

`algorithm for each step`
1. Make pipeline
    - standard data, model
2. GridSearchCV by 10 fold
    - make parameter set
    - reference google and sklearn documents
3. Test
    - best_esitimater is model that construct best hyper parameter set
4. Analysis result
    - use confusion matrix
    - use adj r^2 square that is model Perfection

In [5]:
# Data split
'''
target과 data 나누는
data slicing 필요
문자를 one hot encoding으로 받아야함
'''
from sklearn.model_selection import train_test_split
x_train, y_train, x_test, y_test = train_test_split(sample.data, sample.target, random_state = 1)

In [ ]:
# package
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score, precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix

# Make adj R2 score
def ar2_score(y_test, y_pred, p, n_samples):
    r2 = r2_score(y_test,y_pred)
    result = 1 - (1 - r2)*(n_samples-1)/(n_samples-p-1)
    return result

In [ ]:
# Linear regression train
'''
hyper parameter 선정필요, grid search, kv=5 or 10
'''
# pipeline 생성 데이터를 정규화 시키고, 
pipe_svc = Pipeline([('scl', StandardScaler()),('lr',LinearRegression())])
'''
# hyper parameter set

implement this area!

param_grid = [ 

]
'''
gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='r2', cv=10, n_jobs = -1)
gs.fit( )
best_params = gs.best_params_
# 최고의  하이퍼 파라미터 조합 출력
print(best_params)

In [ ]:
# Linear regression test
# 1. confusion matrix -> precision and recall pair
# 2. accuracy r^2 square
best_lr=gs.best_estimator_
y_pred = best_lr.predict( )
score = ar2_score( )
print("adjusted r2 score 는 %2.3f 입니다"%score)
print(classification_report(y_test,y_pred,target_names=[ ]))

In [ ]:
# Rogistic regression train
'''
make pipeline - standard scaler, model
hyper parameter 선정필요, 
grid search, 
kv=5 or 10
'''

'''
hyper parameter 선정필요, grid search, kv=5 or 10
'''
# pipeline 생성 데이터를 정규화 시키고, 
pipe_svc = Pipeline([('scl', StandardScaler()),('lgr',LogisticRegression())])
'''
# hyper parameter set

implement this area!

param_grid = [ 

]
'''
gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='r2', cv=10, n_jobs = -1)
gs.fit( )
best_params = gs.best_params_
# 최고의  하이퍼 파라미터 조합 출력
print(best_params)

In [ ]:
# Rogistic regression test
# 1. confusion matrix -> precision and recall pair
# 2. accuracy  r^2 square
best_lr=gs.best_estimator_
y_pred = best_lr.predict( )
score = ar2_score( )
print("adjusted r2 score 는 %2.3f 입니다"%score)
print(classification_report(y_test,y_pred,target_names=[ ]))

In [ ]:
# Neural Network  train

'''
hyper parameter 선정필요, 
grid search,
kv=5 or 10
'''
nn = MLPRegressor()
nn.fit()
regr.predict()

In [ ]:
# Neural Network test
# 1. confusion matrix -> precision and recall pair
# 2. accuracy  r^2 square
best_lr=gs.best_estimator_
y_pred = best_lr.predict( )
score = ar2_score( )
print("adjusted r2 score 는 %2.3f 입니다"%score)
print(classification_report(y_test,y_pred,target_names=[ ]))